# BERT training on raw data

This script fine-tunes the BERT model to the given dataset.
You have to provide two variables:

- data_path: Location of the data. Expects a csv file with a "Comment_text" and a "Hateful_or_not" column.
- model_name: Location where the model will be saved.

Finetuning a gigantic model like BERT takes a lot of time. It makes sense to use cloud computing resources to run this script.
Make sure to exclude the test set from the BERT finetuning process, otherwise you'll have data leakage.

## Preparatory steps



### Upload necessary files to the working directory

The data file as .csv and the .py script need to be in /content/ (which is the standard working directory for colab).

Upload the data and bert_functions_modified.py first by executing the following code twice and chossing the 2 necessary files:

In [ ]:
# access data from the drive
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving training_data_hate_speech.csv to training_data_hate_speech.csv


### Installing necessary packages

Install the necessary bert packages and Tensorflow 1.15.2

In [ ]:
!pip uninstall gast
!pip install gast==0.2.2
!pip install sentencepiece
!pip install bert-tensorflow==1.0.1
!pip install pandas==0.24

TensorFlow 1.x selected.
Uninstalling gast-0.3.3:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/gast-0.3.3.dist-info/*
    /usr/local/lib/python3.7/dist-packages/gast/*
Proceed (y/n)? y
  Successfully uninstalled gast-0.3.3
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7540 sha256=e6c4c7f2226ff21171af1aa106eecd8166156822dc40a8f3e2557dd743271680
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
     |████████████████████████████████| 1.2MB 8.0MB/s 
     |████████████████████████████████| 71kB 5.3MB/s 


In [ ]:
%tensorflow_version 1.x

In [ ]:
# for importing modules in .py files, insert the working directory
# import sys
# sys.path.insert(0,'/content/drive/My Drive/AIandLaw/CodeAndModel/')

## Preparing the dataset

In [ ]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.model_selection import train_test_split
from bert_functions_modified import *

### User Input

In [ ]:
data_path = "training_data_hate_speech.csv"
model_name = "bert_model_modified"

### Initialize data and session

In [ ]:
# Load and split data into training and test data
data = pd.read_csv(data_path, sep=',')
X_train, X_test = train_test_split(data, test_size=0.25, random_state=25)
print(X_train.shape)
print(X_test.shape)

# Initialize session
sess = tf.Session()

# Params for bert model and tokenization
bert_path = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"
# max comment length for padding
max_seq_length = 256

(146299, 3)
(48767, 3)


In [ ]:
# Create datasets (Only take up to max_seq_length words for memory)
train_text = X_train['Comment_text'].tolist()
train_text = [' '.join(str(t).split()[0:max_seq_length]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = X_train['Hateful_or_not'].tolist()

test_text = X_test['Comment_text'].tolist()
test_text = [' '.join(t.split()[0:max_seq_length]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = X_test['Hateful_or_not'].tolist()

### Prepare input features from dataset

In [ ]:
# Instantiate tokenizer
tokenizer = create_tokenizer_from_hub_module(bert_path, sess)

# Convert data to InputExample format
train_examples = convert_text_to_examples(train_text, train_label)
test_examples = convert_text_to_examples(test_text, test_label)

# Convert to features
(train_input_ids, train_input_masks, train_segment_ids, train_labels) = convert_examples_to_features(tokenizer, train_examples, max_seq_length=max_seq_length)
(test_input_ids, test_input_masks, test_segment_ids, test_labels) = convert_examples_to_features(tokenizer, test_examples, max_seq_length=max_seq_length)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


## Build and train model

Note: For the next line to work, some modificiations in bert_functions.py were needed.

In particular,
"class BertLayer(tf.layers.Layer):"
was changed to
"class BertLayer(tf.keras.layers.Layer):"

And a custom get_config function was added to the BertLayer class.

The filed was renamed to bert_functions_modified.py

More details about the hyperparameters and layers can be found in bert_functions_modified.py

In [ ]:
bert_model = build_model(max_seq_length, bert_path=bert_path)

# Instantiate variables
initialize_vars(sess)

bert_model.fit(
    [train_input_ids, train_input_masks, train_segment_ids],
    train_labels,
    validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
    epochs=1,
    batch_size=128
)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_layer (BertLayer)          (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]          

Train on 146299 samples, validate on 48767 samples
146299/146299 [==============================] - 4395s 30ms/sample - loss: 0.1363 - acc: 0.9456 - val_loss: 0.1116 - val_acc: 0.9561


In [ ]:
bert_model.save(model_name + ".h5")